In [96]:
import os
import numpy as np
import pandas as pd



In [97]:
INPUT_FOLDER = "enrich"
OUTPUT_FOLDER = ""

In [98]:
key = "us_shareproce_joined_companies"

data = pd.read_csv(f"data/{INPUT_FOLDER}/{key}.csv")
data.info(show_counts=True)
data['Date'] = pd.to_datetime(data['Date'])

data.set_index('Date', inplace=True)  # Set Date as the index


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5380108 entries, 0 to 5380107
Data columns (total 11 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   Date          5380108 non-null  object 
 1   Industry      5380108 non-null  object 
 2   Sector        5380108 non-null  object 
 3   Ticker        5380108 non-null  object 
 4   Open          5380108 non-null  float64
 5   High          5380108 non-null  float64
 6   Low           5380108 non-null  float64
 7   Close         5380108 non-null  float64
 8   Volume        5380108 non-null  int64  
 9   Dividend      5380108 non-null  float64
 10  Company Name  5380108 non-null  object 
dtypes: float64(5), int64(1), object(5)
memory usage: 451.5+ MB


In [99]:
data.head()

,Industry,Sector,Ticker,Open,High,Low,Close,Volume,Dividend,Company Name
Date,,,,,,,,,,
2019-04-11,Industrial Products,Industrials,AAON,29.90,30.27,29.75,30.08,164812,0.0,AAON INC
2019-04-11,REITs,Real Estate,ESRT,15.89,15.94,15.74,15.83,354551,0.0,"Empire State Realty Trust, Inc."
2019-04-11,Banks,Financial Services,HWC,42.81,43.09,42.24,42.59,337095,0.0,HANCOCK WHITNEY CORP
2019-04-11,Banks,Financial Services,IBCP,22.15,22.34,22.05,22.09,45879,0.0,Independent Bank Corporation
2019-04-11,Banks,Financial Services,IBN,11.25,11.25,11.16,11.19,3291052,0.0,ICICI Bank Limited


In [100]:
data['Sector'].unique()

array(['Industrials', 'Real Estate', 'Financial Services', 'Technology',
       'Basic Materials', 'Utilities', 'Business Services', 'Energy',
       'Healthcare', 'Consumer Defensive', 'Consumer Cyclical', 'Other'],
      dtype=object)

In [101]:
data=data[data['Sector']=='Technology']

In [102]:
data=data[data['Volume']>100000000]

In [103]:
len(data['Ticker'].unique())
data['Ticker'].unique()

array(['NVDA', 'AAPL', 'QCOM', 'TWTR', 'SNAP', 'GOOG', 'AMD', 'AVGO',
       'MU', 'CSCO', 'WHEN', 'DPLS', 'HMNY', 'NOK', 'ANET', 'MVIS',
       'HYSR', 'INTC', 'SCKT', 'PINS', 'ADT', 'SUNW', 'PLTR', 'PPSI',
       'AITX', 'BB', 'DDD', 'BSQR', 'SPRT', 'NUAN', 'LEDS', 'BLIN',
       'TROO', 'AEHR', 'ZNGA', 'RCAT', 'PALT', 'ATVI', 'META', 'CASA',
       'RBLX', 'AVYA', 'BEAT', 'AI', 'SQ', 'PANW', 'SMCI', 'MARK', 'SOUN',
       'AXTI'], dtype=object)

In [104]:
selected_tickers = ['NVDA', 'BB', 'SNAP', 'HYSR', 'AMD', 'META']
data_filtered = data[data['Ticker'].isin(selected_tickers)]


In [105]:
#data_filtered[data_filtered.index>"2024.01.01"]

In [106]:
# Ensure data is sorted by date
data_filtered = data_filtered.sort_values(by=['Ticker', 'Date'])

# Create binary target: 1 if next day's Close is higher, else 0
data_filtered['Target'] = 0  # Default: Hold
data_filtered.loc[data_filtered['Close'].shift(-1) > data_filtered['Close'] * 1.02, 'Target'] = 1  # Buy if price up >2%
data_filtered.loc[data_filtered['Close'].shift(-1) < data_filtered['Close'] * 0.98, 'Target'] = 2  # Sell if price down >2%

data_filtered.dropna(inplace=True)

# Drop NaN values created by shift
data_filtered.dropna(inplace=True)
print(data_filtered['Target'].value_counts(normalize=True))


Target
0    0.489056
1    0.287278
2    0.223666
Name: proportion, dtype: float64


In [107]:
# Moving Averages (Trend Indicators)
data_filtered['MA_10'] = data_filtered.groupby('Ticker')['Close'].transform(lambda x: x.rolling(10).mean())
data_filtered['MA_50'] = data_filtered.groupby('Ticker')['Close'].transform(lambda x: x.rolling(50).mean())

# Volatility Indicator
data_filtered['Volatility'] = data_filtered.groupby('Ticker')['Close'].transform(lambda x: x.rolling(10).std())

# Relative Strength Index (Momentum Indicator)
def compute_rsi(series, window=14):
    delta = series.diff()
    gain = delta.where(delta > 0, 0).rolling(window=window).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=window).mean()
    rs = gain / (loss + 1e-10)  # Avoid division by zero
    return 100 - (100 / (1 + rs))

data_filtered['RSI'] = data_filtered.groupby('Ticker')['Close'].transform(lambda x: compute_rsi(x))

# Bollinger Bands (Market Volatility)
data_filtered['BB_Upper'] = data_filtered['MA_10'] + (2 * data_filtered['Volatility'])
data_filtered['BB_Lower'] = data_filtered['MA_10'] - (2 * data_filtered['Volatility'])

data_filtered.dropna(inplace=True)


NVIDIA Boosting

In [108]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score

# Define hyperparameter grid
param_grid = {
    'n_estimators': [100, 300, 500],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.03, 0.1],
    
}

# Filter data for NVDA
stock_data = data_filtered[data_filtered['Ticker'] == 'AMD']
features = ['Open', 'High', 'Low', 'Volume','MA_10','MA_50', 'Volatility', 'RSI', 'BB_Upper', 'BB_Lower']

# Train-Test Split (Time-Based)
split = int(len(stock_data) * 0.8)
X_train, X_test = stock_data[features].iloc[:split], stock_data[features].iloc[split:]
y_train, y_test = stock_data['Target'].iloc[:split], stock_data['Target'].iloc[split:]


model = XGBClassifier(objective='multi:softmax', num_class=3, random_state=42)

# Perform Grid Search for Best Hyperparameters
grid_search = GridSearchCV(model, param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Train Final Model with Best Hyperparameters
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

# Make Predictions
y_pred = best_model.predict(X_test)

# Evaluate Model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')


print(classification_report(y_test, y_pred))
print(grid_search.best_params_)

Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.0s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=300; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=300; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=6, n_estimators=100; total time=   0.1s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=300; total time=   0.2s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=500; total time=   0.2s
[CV] END ..learning_rate=0.01, max_depth=6, n_estimators=300; total time=   0.2s
[CV] END ..learning_rate=0.01, max_depth=6, n_estimators=300; total time=   0.2s
[CV] END ..learning_rate=0.01, max_depth=6, n_estimators=300; total time=   0.2s
[CV] END ..learning_rate=0.01, max_depth=9, n_es

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-pa

In [109]:
accuracy

0.3333333333333333

In [110]:
import joblib

# Save the trained model
joblib.dump(best_model, "ml_model.pkl")

['ml_model.pkl']

**AAPL Boosting**